# Train Various Models on MNIST using kubeflow and seldon-core

Using:

 * kubeflow
 * seldon-core
 
The example will be the MNIST handwriiten digit classification task.

![MNIST](mnist.png "MNIST Digits")

### Dependencies

  * Argo

# Setup


In [1]:
!kubectl config set-context $(kubectl config current-context) --namespace=kubeflow

Context "my-kubeflow" modified.


# Tensorflow Model
 A simple neural network in Tensorflow.

### Training
 * Create image from source
 * Run training
 

Run with:
  * ``` -p build-push-image=true``` to build image and push to repo, needed extra params:
    * ``` -p version=<version>``` create ```<version>``` of model
    * ``` -p github-user=<github-user>``` to download example-seldon source from ```<github-user>``` account
    * ``` -p github-revision=<revision>``` to use the github branch ```<revision>```
    * ``` -p docker-user=<docker-user>``` to use Docker user ```<docker-user>``` to push image to. Needs docker credentials in secret as described in README.

In [5]:
!pygmentize ../workflows/training-tf-mnist-workflow.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: kubeflow-tf-train-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.1
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master
    - name: docker-user
      value: seldonio
    - name: tfjob-version-hack
      value: 1
    - name: build-push-image
      value: false
  volumes:
  - name: my-secret-vol
    secret:
      secretName: docker-credentials     # name of an existing k8s secret
  templates:
  - name: workflow
    steps:
    - - name: build-push 
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: train
        template: tfjob
  - name: build-and-push
    inputs:
      artifacts:
      - name: argo-source
        path: /src/example-seldon
        git:
          repo: https://github.com/{{workflow.parameters.github-user}}/example-seldon.git
          revision: "{{workfl

In [8]:
!argo submit ../workflows/training-tf-mnist-workflow.yaml -p tfjob-version-hack=1

Name:                kubeflow-tf-train-wx8xq
Namespace:           kubeflow
ServiceAccount:      default
Status:              Pending
Created:             Sat Jan 19 17:55:00 +0000 (now)
Parameters:          
  tfjob-version-hack: 1
  version:           0.1
  github-user:       kubeflow
  github-revision:   master
  docker-user:       seldonio
  build-push-image:  false


In [3]:
!argo list

NAME                      STATUS    AGE   DURATION
kubeflow-tf-train-2nqgz   Running   2s    2s 


### Runtime Image

Run with:
  * ``` -p build-push-image=true``` to build image and push to repo, needed extra params:
    * ``` -p version=<version>``` create ```<version>``` of model
    * ``` -p github-user=<github-user>``` to download example-seldon source from ```<github-user>``` account
    * ``` -p github-revision=<revision>``` to use the github branch ```<revision>```
    * ``` -p docker-user=<docker-user>``` to use Docker user ```<docker-user>``` to push image to. Needs docker credentials in secret as described in README.
  * ``` -p deploy-model=true``` to deploy model

In [5]:
!pygmentize ../workflows/serving-tf-mnist-workflow.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: seldon-tf-deploy-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.1
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master
    - name: docker-user
      value: seldonio
    - name: build-push-image
      value: false
    - name: deploy-model
      value: false
  volumes:
  - name: my-secret-vol
    secret:
      secretName: docker-credentials     # name of an existing k8s secret
  templates:
  - name: workflow
    steps:
    - - name: build-push 
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: serve
        template: seldon
        when: "{{workflow.parameters.deploy-model}} == true"	
  - name: build-and-push
    inputs:
      artifacts:
      - name: argo-source
        path: /src/example-seldon
        git:
          repo: https://github.com/{{workflow.parameters.gi

In [9]:
!argo submit ../workflows/serving-tf-mnist-workflow.yaml

Name:                seldon-tf-deploy-ckr59
Namespace:           kubeflow
ServiceAccount:      default
Status:              Pending
Created:             Sat Jan 19 18:50:11 +0000 (now)
Parameters:          
  version:           0.1
  github-user:       kubeflow
  github-revision:   master
  docker-user:       seldonio
  build-push-image:  false
  deploy-model:      false


In [7]:
!argo list

NAME                      STATUS      AGE   DURATION
kubeflow-tf-train-kpsvp   Running     2s    2s 
seldon-tf-deploy-mvwwl    Succeeded   1s    0s


# Sklearn Model
A Random forest in sklearn.

### Training

 * For options see above Tensorflow example

In [8]:
!pygmentize ../workflows/training-sk-mnist-workflow.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: kubeflow-sk-train-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.2
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master      
    - name: docker-user
      value: seldonio
    - name: build-push-image
      value: false      
  volumes:
  - name: my-secret-vol
    secret:
      secretName: docker-credentials     # name of an existing k8s secret
  templates:
  - name: workflow
    steps:
    - - name: build-push 
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: train
        template: tfjob
  - name: build-and-push
    inputs:
      artifacts:
      - name: argo-source
        path: /src/example-seldon
        git:
          repo: https://github.com/{{workflow.parameters.github-user}}/example-seldon.git
          revision: "{{workflow.parameters.github-revision}}"  

In [10]:
!argo submit ../workflows/training-sk-mnist-workflow.yaml

Name:                kubeflow-sk-train-ch479
Namespace:           kubeflow
ServiceAccount:      default
Status:              Pending
Created:             Sat Jan 19 18:50:40 +0000 (1 second ago)
Parameters:          
  version:           0.2
  github-user:       kubeflow
  github-revision:   master
  docker-user:       seldonio
  build-push-image:  false


In [10]:
!argo list

NAME                      STATUS      AGE   DURATION
kubeflow-sk-train-z8x7n   Running     0s    0s
kubeflow-tf-train-kpsvp   Running     3s    3s 
seldon-tf-deploy-mvwwl    Succeeded   2s    0s


### Runtime Image
 * For options see above Tensorflow example

In [11]:
!pygmentize ../workflows/serving-sk-mnist-workflow.yaml

# This example demonstrates the use of a git repo as a hard-wired 
# input artifact. The argo repo is cloned to its target destination
# at '/src' for the main container to consume.
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: seldon-sk-deploy-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.1
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master
    - name: docker-user
      value: seldonio
    - name: build-push-image
      value: false
    - name: deploy-model
      value: false
  volumes:
  - name: my-secret-vol
    secret:
      secretName: docker-credentials     # name of an existing k8s secret
  templates:
  - name: workflow
    steps:
    - - name: build-push 
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: serve
        template: seldon
        when: "{{workflow.parameters.deploy-model}} == true"		
  -

In [12]:
!argo submit ../workflows/serving-sk-mnist-workflow.yaml

Name:                seldon-sk-deploy-r8ghl
Namespace:           kubeflow
ServiceAccount:      default
Status:              Pending
Created:             Mon Nov 26 16:40:56 +0000 (now)
Parameters:          
  version:           0.1
  github-user:       kubeflow
  github-revision:   master
  docker-user:       seldonio
  build-push-image:  false
  deploy-model:      false


In [13]:
!argo list

NAME                      STATUS      AGE   DURATION
kubeflow-sk-train-z8x7n   Running     2s    2s 
kubeflow-tf-train-kpsvp   Running     5s    5s 
seldon-sk-deploy-r8ghl    Succeeded   1s    0s
seldon-tf-deploy-mvwwl    Succeeded   4s    0s


# R Model
A partial least squares model in R.

### Training

 * For options see above Tensorflow example

In [14]:
!pygmentize ../workflows/training-r-mnist-workflow.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: kubeflow-r-train-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.1
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master      
    - name: docker-user
      value: seldonio
    - name: build-push-image
      value: false      
  volumes:
  - name: my-secret-vol
    secret:
      secretName: docker-credentials     # name of an existing k8s secret
  templates:
  - name: workflow
    steps:
    - - name: build-push 
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: train
        template: tfjob
  - name: build-and-push
    inputs:
      artifacts:
      - name: argo-source
        path: /src/example-seldon
        git:
          repo: https://github.com/{{workflow.parameters.github-user}}/example-seldon.git
          revision: "{{workflow.parameters.github-revision}}"   

In [11]:
!argo submit ../workflows/training-r-mnist-workflow.yaml

Name:                kubeflow-r-train-q72r6
Namespace:           kubeflow
ServiceAccount:      default
Status:              Pending
Created:             Sun Jan 20 09:08:44 +0000 (now)
Parameters:          
  version:           0.1
  github-user:       kubeflow
  github-revision:   master
  docker-user:       seldonio
  build-push-image:  false


In [16]:
!argo list

NAME                      STATUS      AGE   DURATION
kubeflow-r-train-7fwb6    Running     0s    0s
kubeflow-sk-train-z8x7n   Running     3s    3s 
kubeflow-tf-train-kpsvp   Running     6s    6s 
seldon-sk-deploy-r8ghl    Succeeded   2s    0s
seldon-tf-deploy-mvwwl    Succeeded   5s    0s


### Runtime Image
 * For options see above Tensorflow example

In [17]:
!pygmentize ../workflows/serving-r-mnist-workflow.yaml

# This example demonstrates the use of a git repo as a hard-wired 
# input artifact. The argo repo is cloned to its target destination
# at '/src' for the main container to consume.
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: seldon-r-deploy-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.1
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master
    - name: docker-user
      value: seldonio
    - name: build-push-image
      value: false
    - name: deploy-model
      value: false
  volumes:
  - name: my-secret-vol
    secret:
      secretName: docker-credentials     # name of an existing k8s secret
  templates:
  - name: workflow
    steps:
    - - name: build-push 
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: serve
        template: seldon
        when: "{{workflow.parameters.deploy-model}} == true"			
  -

In [18]:
!argo submit ../workflows/serving-r-mnist-workflow.yaml

Name:                seldon-r-deploy-hvww9
Namespace:           kubeflow
ServiceAccount:      default
Status:              Pending
Created:             Mon Nov 26 16:40:59 +0000 (now)
Parameters:          
  version:           0.1
  github-user:       kubeflow
  github-revision:   master
  docker-user:       seldonio
  build-push-image:  false
  deploy-model:      false


In [19]:
!argo list

NAME                      STATUS      AGE   DURATION
kubeflow-r-train-7fwb6    Running     2s    2s 
kubeflow-sk-train-z8x7n   Running     5s    5s 
kubeflow-tf-train-kpsvp   Running     8s    8s 
seldon-r-deploy-hvww9     Succeeded   1s    0s
seldon-sk-deploy-r8ghl    Succeeded   4s    0s
seldon-tf-deploy-mvwwl    Succeeded   7s    0s
